In [1]:
import numpy as np
import pandas as pd
pd.options.plotting.backend = 'plotly'
from my_stuff import MufexKeys
from quantfreedom.enums import *
from quantfreedom.exchanges.mufex_exchange.mufex import Mufex
from quantfreedom.indicators.indicators import ema_calc, rsi_calc
from quantfreedom.exchanges.exchange import Exchange
from numba import njit
mufex_main = Mufex(
    api_key=MufexKeys.api_key,
    secret_key=MufexKeys.secret_key,
    use_test_net=False,
)

%load_ext autoreload
%autoreload 2

In [2]:
candles = mufex_main.get_candles(
    symbol="BTCUSDT", timeframe='5m', candles_to_dl=300)

It took 00 mins and 01 seconds to download 300 candles


In [3]:
b_candles = Exchange(api_key='d', secret_key='s', use_test_net=False).get_binance_futures_candles(symbol='BTCUSDT', timeframe='5m', candles_to_dl=300)

It took 00 mins and 02 seconds to download 300 candles


In [4]:
closes = candles[:,4]
b_closes = b_candles[:,4]

In [6]:
np.around(rsi_calc(closes, 14),4)

array([    nan,     nan,     nan,     nan,     nan,     nan,     nan,
           nan,     nan,     nan,     nan,     nan,     nan,     nan,
       70.1243, 68.9809, 62.6044, 64.5907, 65.8271, 59.5446, 53.0821,
       59.5768, 63.2341, 65.0184, 62.164 , 56.1511, 54.6476, 56.8037,
       54.7787, 48.3771, 49.1209, 51.3003, 56.7993, 48.2934, 49.4892,
       54.0412, 50.0209, 51.3651, 56.3913, 55.7678, 60.1132, 57.079 ,
       59.2238, 54.5723, 54.9776, 57.0674, 60.6157, 54.5572, 54.013 ,
       57.495 , 67.4209, 80.1479, 81.2807, 68.1506, 68.5046, 73.1384,
       69.932 , 60.6869, 59.1405, 60.4459, 63.1444, 60.126 , 59.5625,
       61.6007, 62.0359, 60.9938, 57.6182, 57.043 , 53.7515, 56.3998,
       57.5846, 55.6747, 58.2828, 62.6204, 60.9924, 65.3079, 64.9065,
       70.3529, 71.1767, 72.5183, 75.7115, 69.3498, 62.2358, 53.6538,
       60.1715, 60.602 , 54.7007, 53.1951, 50.4782, 55.1046, 56.2394,
       57.0192, 51.8363, 52.679 , 54.1662, 48.62  , 47.5163, 47.1235,
       43.1535, 44.9